# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TESTING_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_NONSPAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'
VOCAB_SIZE = 2500


# Read and Load features from .txt files to np.array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter = ' ', dtype = int)

In [4]:
sparse_test_data = np.loadtxt(TESTING_DATA_FILE, delimiter = ' ', dtype = int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2032,    0,    1],
       [5795, 2034,    0,    1],
       [5795, 2269,    0,    1],
       [5795, 2329,    0,    1],
       [5795, 2393,    0,    1]])

In [7]:
print('Rows in training file ', sparse_train_data.shape[0])
print('Rows in testing file ', sparse_test_data.shape[0])

Rows in training file  257918
Rows in testing file  117574


In [8]:
print('Num of emails in train file',np.unique(sparse_train_data[: , 0]).size)

Num of emails in train file 4013


## Empty Dataframe, to fill later

In [9]:
column_names = ['DOC_ID', 'CATEGORY'] + list(range(0,VOCAB_SIZE))
column_names[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [11]:
full_train_data = pd.DataFrame(index=index_names, columns= column_names)

In [12]:
full_train_data.fillna(value=0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creating Full Matrix from Sparse Matrix

In [13]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx = 3):
    """
    Form a full matrix from a sparse matrix and return a pd DataFrame. 
    Keyword args:
    sparse_matrix --- numpy array
    nr_words --- size of vocabulary, total number of tokens
    doc_idx --- position of doc_id in sparse matrix
    word_idx--- position of word_id in sparse matrix
    cat_idx--- position of category id in sparse matrix
    freq_id--- position of occurence id in the sparse matrix
    """
    
    column_names = ['DOC_ID', 'CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns= column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id= sparse_matrix[i][word_idx]
        label =  sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID']=doc_nr
        full_matrix.at[doc_nr, 'CATEGORY']=label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
        
    return full_matrix
    

In [14]:
%%time 
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 23.9 s


In [15]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Baye's Model

### Calculating probability of Spam

In [16]:
full_train_data.CATEGORY.sum()

1248

In [17]:
full_train_data.CATEGORY.size

4013

In [18]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam

0.310989284824321

### Total number of words/ tokens

In [19]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [21]:
email_lengths.head()

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [22]:
total_wc = email_lengths.sum()
total_wc

428371

## Number of tokens in Spam and Non-Spam Emails

In [23]:
spam_lengths = email_lengths[full_train_data.CATEGORY==1]
spam_lengths.shape

(1248,)

In [24]:
spam_wc = spam_lengths.sum()
spam_wc

175883

In [25]:
nonspam_lengths = email_lengths[full_train_data.CATEGORY==0]
nonspam_lengths.shape


(2765,)

In [26]:
nonspam_wc = nonspam_lengths.sum()
nonspam_wc

252488

## Summing Tokens in Spam

In [27]:
full_train_features.shape

(4013, 2500)

In [28]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

## Doing same for nonspam

In [30]:
train_nonspam_tokens = full_train_features.loc[full_train_data.CATEGORY==0]
train_nonspam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
summed_nonspam_tokens = train_nonspam_tokens.sum(axis = 0) + 1
summed_nonspam_tokens.shape

(2500,)

## Probability (Token | Spam)

In [32]:
prob_token_spam = summed_spam_tokens/ (spam_wc+VOCAB_SIZE)
prob_token_spam.shape

(2500,)

In [33]:
prob_token_spam.sum()

1.0

## Probability (Token | Non-Spam)

In [34]:
prob_token_nonspam = summed_nonspam_tokens/ (nonspam_wc+VOCAB_SIZE)
prob_token_nonspam.shape

(2500,)

In [35]:
prob_token_nonspam.sum()

0.9999999999999999

## Probability(Token)

In [36]:
prob_tokens_all = full_train_features.sum(axis=0)/total_wc
prob_tokens_all.sum()

1.0

## Saving our Model

In [37]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_token_spam)
np.savetxt(TOKEN_NONSPAM_PROB_FILE, prob_token_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

In [38]:
%%time
full_test_data=make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 7.8 s


In [39]:
x_test=full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [40]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, x_test)

In [42]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam

0.310989284824321

In [44]:
prob_nonspam =1- full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size

In [45]:
prob_nonspam

0.689010715175679